In [1]:
import numpy as np
import tvm

In [2]:
tgt_host="llvm"
# Change it to respective GPU if gpu is enabled Ex: cuda, opencl
tgt="opencl"

In [3]:
n = tvm.var("n")
A = tvm.placeholder((n,), name='A')
B = tvm.placeholder((n,), name='B')
C = tvm.compute(A.shape, lambda i: A[i] + B[i], name="C")
print(type(C))

<class 'tvm.tensor.Tensor'>


In [4]:
s = tvm.create_schedule(C.op)

In [5]:
bx, tx = s[C].split(C.op.axis[0], factor=64)

In [6]:
if tgt == "cuda" or tgt.startswith('opencl'):
  s[C].bind(bx, tvm.thread_axis("blockIdx.x"))
  s[C].bind(tx, tvm.thread_axis("threadIdx.x"))

In [7]:
fadd = tvm.build(s, [A, B, C], tgt, target_host=tgt_host, name="myadd")

In [8]:
if tgt == "cuda" or tgt.startswith('opencl'):
    dev_module = fadd.imported_modules[0]
    #host_module= fadd.imported_modules[1]
    print("-----Device code-----")
    print(dev_module.get_source())
    #print("-----Host code-----")
    #print(host_module.get_source())

-----Device code-----
__kernel void myadd_kernel0(__global float* restrict C, __global float* restrict A, __global float* restrict B, int n) {
  if (((int)get_group_id(0)) < (n / 64)) {
    C[((((int)get_group_id(0)) * 64) + ((int)get_local_id(0)))] = (A[((((int)get_group_id(0)) * 64) + ((int)get_local_id(0)))] + B[((((int)get_group_id(0)) * 64) + ((int)get_local_id(0)))]);
  } else {
    if ((((int)get_group_id(0)) * 64) < (n - ((int)get_local_id(0)))) {
      C[((((int)get_group_id(0)) * 64) + ((int)get_local_id(0)))] = (A[((((int)get_group_id(0)) * 64) + ((int)get_local_id(0)))] + B[((((int)get_group_id(0)) * 64) + ((int)get_local_id(0)))]);
    }
  }
}


